Copyright Preferred Computational Chemistry, Inc. as contributors to Matlantis contrib project

In [ ]:
!pip install pfp_api_client
!pip install pandas tqdm matplotlib seaborn optuna sklearn ase

# # 初回使用時のみ、ライブラリのインストールをお願いします。

In [1]:
import pathlib
EXAMPLE_DIR = pathlib.Path("__file__").resolve().parent
INPUT_DIR = EXAMPLE_DIR / "input"
OUTPUT_DIR = EXAMPLE_DIR / "output"
OUTPUT_DIR.mkdir(exist_ok=True)

In [2]:
# 汎用モジュール
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from IPython.display import Image, display_png
import ipywidgets as widgets
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.widgets import Slider
from matplotlib.animation import PillowWriter
import seaborn as sns
import math
import optuna
import nglview as nv
import os,sys,csv,glob,shutil,re,time
from time import perf_counter
from joblib import Parallel, delayed

# sklearn
from sklearn.metrics import mean_absolute_error

import ase
from ase.visualize import view
from ase.optimize import BFGS
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, ExpCellFilter

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md import MDLogger
from ase import Atoms
from ase.io import read, write
from ase.io import Trajectory
from ase import units

from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator

estimator = Estimator()
calculator = ASECalculator(estimator)

In [3]:
def myopt(m,sn = 10,constraintatoms=[],cbonds=[]):
    fa = FixAtoms(indices=constraintatoms)
    fb = FixBondLengths(cbonds,tolerance=1e-5,)
    m.set_constraint([fa,fb])
    m.set_calculator(calculator)
    maxf = np.sqrt(((m.get_forces())**2).sum(axis=1).max())
    print("ini   pot:{:.4f},maxforce:{:.4f}".format(m.get_potential_energy(),maxf))
    de = -1 
    s = 1
    ita = 50
    while ( de  < -0.001 or de > 0.001 ) and s <= sn :
        opt = BFGS(m,maxstep=0.04*(0.9**s),logfile=None)
        old  =  m.get_potential_energy() 
        opt.run(fmax=0.0005,steps =ita)
        maxf = np.sqrt(((m.get_forces())**2).sum(axis=1).max())
        de =  m.get_potential_energy()  - old
        print("{} pot:{:.4f},maxforce:{:.4f},delta:{:.4f}".format(s*ita,m.get_potential_energy(),maxf,de))
        s += 1
    return m

def opt_cell_size(m,sn = 10, iter_count = False): # m:Atomsオブジェクト
    m.set_constraint() # clear constraint
    m.set_calculator(calculator)
    maxf = np.sqrt(((m.get_forces())**2).sum(axis=1).max()) # √(fx^2 + fy^2 + fz^2)の一番大きいものを取得
    ucf = ExpCellFilter(m)
    print("ini   pot:{:.4f},maxforce:{:.4f}".format(m.get_potential_energy(),maxf))
    de = -1 
    s = 1
    ita = 50
    while ( de  < -0.01 or de > 0.01 ) and s <= sn :
        opt = BFGS(ucf,maxstep=0.04*(0.9**s),logfile=None)
        old  =  m.get_potential_energy() 
        opt.run(fmax=0.005,steps =ita)
        maxf = np.sqrt(((m.get_forces())**2).sum(axis=1).max())
        de =  m.get_potential_energy()  - old
        print("{} pot:{:.4f},maxforce:{:.4f},delta:{:.4f}".format(s*ita,m.get_potential_energy(),maxf,de))
        s += 1
    if iter_count == True:
        return m, s*ita
    else:
        return m
    

In [4]:
# A. Jain*, S.P. Ong*, G. Hautier, W. Chen, W.D. Richards, S. Dacek, S. Cholia, D. Gunter, D. Skinner, G. Ceder, K.A. Persson (*=equal contributions)
# The Materials Project: A materials genome approach to accelerating materials innovation
# APL Materials, 2013, 1(1), 011002.
bulk = read(INPUT_DIR / "Li10Ge(PS6)2_mp-696138_computed.cif")
bulk.calc = calculator

print("原子数 =", len(bulk))
print("initial 格子定数 =", bulk.cell.cellpar())

opt_cell_size(bulk)
print ("optimized 格子定数 =", bulk.cell.cellpar())

原子数 = 50
initial 格子定数 = [ 8.589951    8.87954092 12.97496188 91.98415256 90.64261126 90.24910835]
ini   pot:-173.9513,maxforce:0.2045
50 pot:-174.0483,maxforce:0.0261,delta:-0.0970
100 pot:-174.0499,maxforce:0.0044,delta:-0.0016
optimized 格子定数 = [ 8.73160173  8.78787712 13.07286495 90.95841874 90.86646318 90.3552053 ]


In [5]:
bulk = bulk.repeat([2,2,1])
bulk.calc = calculator

In [6]:
v = view(bulk, viewer='ngl')
#v.view.add_representation("ball+stick")
display(v)

In [7]:
os.makedirs(OUTPUT_DIR / "structure/", exist_ok=True)
write(OUTPUT_DIR / "structure/opt_structure.xyz", bulk)

In [8]:
Li_index = [i for i, x in enumerate(bulk.get_chemical_symbols()) if x == 'Li']
print(len(Li_index))

80


In [9]:
temp_list = [423, 523, 623, 723, 823, 923, 973, 1023]

In [10]:
os.makedirs(OUTPUT_DIR / "traj_and_log/", exist_ok=True)

def run_md(i):
    s_time = perf_counter()
    
    estimator = Estimator()
    calculator = ASECalculator(estimator)
    
    t_step = 1     # as fs
    temp = i    # as K
    itrvl = 100
    
    structure = read(f"{OUTPUT_DIR.name}/" + "structure/opt_structure.xyz")
    structure.calc = calculator
    
    MaxwellBoltzmannDistribution(structure, temperature_K=temp)

    dyn = Langevin(
                structure,
                t_step * units.fs,
                temperature_K=temp,
                friction=0.02,
                trajectory=f"{OUTPUT_DIR.name}/" + "traj_and_log/MD_"+str(i).zfill(4)+".traj",
                loginterval=itrvl,
                append_trajectory=False,
                )

    dyn.attach(MDLogger(dyn, structure, f"{OUTPUT_DIR.name}/" + "traj_and_log/MD_"+str(i).zfill(4)+".log", header=False, stress=False,
               peratom=True, mode="w"), interval=itrvl)

    dyn.run(500000)
    proctime = perf_counter() - s_time

    return([i, proctime/3600])

In [11]:
results = Parallel(n_jobs=len(temp_list), verbose=1)(delayed(run_md)(i) for i in temp_list)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/home/jovyan/.local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed: 855.6min remaining: 2566.9min
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed: 872.6min finished


In [12]:
# 計算時間
print(results)

[[423, 14.271925339630833], [523, 14.259997109079167], [623, 14.230207275385276], [723, 14.47737348962167], [823, 14.5348415822825], [923, 14.29167752428389], [973, 14.296882973854165], [1023, 14.458852979424446]]
